In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import joblib
import optuna
import json
import sys

from sklearn.metrics import f1_score

# gradient boosting libraries
import xgboost as xgb
from catboost import CatBoostClassifier

# --- TabM stuff ---
# from f2_preprocessor import Preprocessor as F2Preprocessor
from TabM_save_load_new import TabM_save_load   # your class file with save/load logic
from tabm import TabM
from rtdl_num_embeddings import LinearReLUEmbeddings

# --- CatBoost preprocessor (cc5) ---
from cc5_preprocessor import Preprocessor as CC5Preprocessor

# --- Robust Preprocessor (SW_preprocessor) ---
from SW_preprocessor import enhanced_feature_engineer, RobustPreprocessor

device = 'cpu'

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("data/Training_TriGuard.csv")
df = df.dropna(subset=['subrogation'])
target_col = "subrogation"
y_train = df[target_col].astype(int).to_numpy()

real_test = pd.read_csv("data/Testing_TriGuard.csv")

In [3]:
def build_model(n_num_features: int,
                cat_cardinalities,
                trial: optuna.Trial) -> nn.Module:
    # EDIT: tighten but beef up architecture search space
    n_blocks = trial.suggest_int("n_blocks", 3, 5)                 # was 2–5
    d_block  = trial.suggest_int("d_block", 384, 1024, log=True)   # was 256–1024
    dropout  = trial.suggest_float("dropout", 0.0, 0.4)
    k        = trial.suggest_int("k", 8, 32, step=8)               # keep 8–32

    num_emb = LinearReLUEmbeddings(n_num_features)

    model = TabM.make(
        n_num_features=n_num_features,
        num_embeddings=num_emb,
        cat_cardinalities=cat_cardinalities,
        d_out=1,
        n_blocks=n_blocks,
        d_block=d_block,
        dropout=dropout,
        k=k,
    )

    return model.to(device)

## Loading TabM

In [4]:
tabm_pipeline = TabM_save_load.load(
    path="models/tabm_full_pipeline_5938",   # <-- adjust if needed
    build_model_fn=build_model,
    device=device,
)

def tabm_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """Use the saved TabM pipeline (with f2_preprocessor) to get P(y=1)."""
    probs, _ = tabm_pipeline.predict(df_input)
    return probs

## Loading XGBoost

In [5]:
sys.modules["__main__"].RobustPreprocessor = RobustPreprocessor

xgb_assets = joblib.load("models/TriGuard_SW14_0.606.pkl")

xgb_preproc           = xgb_assets["preprocessor"]       # can be a function or a transformer
xgb_feature_selector  = xgb_assets["feature_selector"]
xgb_selected_features = xgb_assets["selected_features"]
xgb_model             = xgb_assets["model"]
xgb_best_threshold    = xgb_assets["threshold"]          # we'll ignore this for ensemble


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator SelectFromModel from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [6]:
def xgb_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """
    Apply the XGBoost preprocessing function/object + feature selector + model.
    """
    # If xgb_preproc is a transformer (Pipeline / ColumnTransformer):
    #   X_proc = xgb_preproc.transform(df_input)
    # If it's a function:
    #   X_proc = xgb_preproc(df_input)
    try:
        X_proc = xgb_preproc.transform(df_input)
    except AttributeError:
        X_proc = xgb_preproc(df_input)

    X_sel  = xgb_feature_selector.transform(X_proc)
    proba  = xgb_model.predict_proba(X_sel)[:, 1]
    return proba


## Loading CatBoost

In [7]:
target_col = "subrogation"

# 1. Prepare X, y exactly as in the original notebook
# (You already have df loaded with subrogation.)
df = df.dropna(subset=[target_col])  # just in case

X = df.drop(columns=[target_col]).copy()
y = df[target_col].astype(int).copy()

# 2. Rebuild the original preprocessor in CATBOOST mode
pre = CC5Preprocessor(smoothing_factor=5, mode="catboost")
pre.fit(X, y)

# 3. Transform the FULL training set to get the canonical feature matrix
X_proc = pre.transform(X)

# These are the exact feature columns CatBoost should see
cb_train_cols = X_proc.columns.tolist()

print("Rebuilt CatBoost preprocessor in 'catboost' mode.")
print("Processed train shape:", X_proc.shape)
print("First few columns:", cb_train_cols[:10])

# 4. Load the saved CatBoost model
cb_model = CatBoostClassifier()
cb_model.load_model("models/catboost_mod_f1_0.5937_cv.cbm")  # <-- use your real file name/path


Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Rebuilt CatBoost preprocessor in 'catboost' mode.
Processed train shape: (17999, 161)
First few columns: ['claim_number', 'year_of_born', 'gender', 'email_or_tel_available', 'safety_rating', 'annual_income', 'high_education_ind', 'address_change_ind', 'living_status', 'zip_code']


In [8]:
def cb_predict_proba(df_input: pd.DataFrame) -> np.ndarray:
    """
    Use the original cc5 Preprocessor (catboost mode) + CatBoost model
    to get P(y=1) for any input df (train or test).

    - Drop target column if present.
    - Apply pre.transform (same FE + encoding as training).
    - Reindex to cb_train_cols (fill missing with 0).
    - Call cb_model.predict_proba(...).
    """
    # Drop target if it sneaks in
    X_raw = df_input.drop(columns=[target_col], errors="ignore").copy()

    # Apply the same feature engineering + preprocessing
    X_cb = pre.transform(X_raw)

    # Align columns to what CatBoost was trained on
    X_cb = X_cb.reindex(columns=cb_train_cols, fill_value=0)

    # Predict probabilities
    proba = cb_model.predict_proba(X_cb)[:, 1]
    return proba

## `compute_best_f1()` Helper Function

In [9]:
def compute_best_f1(probs, targets, thresholds=None):
    probs = np.asarray(probs)
    targets = np.asarray(targets).astype(int)

    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)

    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(targets, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_f1, best_t


In [10]:
tabm_train_probs = tabm_predict_proba(df)
xgb_train_probs  = xgb_predict_proba(df)
cb_train_probs   = cb_predict_proba(df)

print("TabM train probs min/mean/max:",
      tabm_train_probs.min(), tabm_train_probs.mean(), tabm_train_probs.max())
print("XGB  train probs min/mean/max:",
      xgb_train_probs.min(), xgb_train_probs.mean(), xgb_train_probs.max())
print("CB   train probs min/mean/max:",
      cb_train_probs.min(), cb_train_probs.mean(), cb_train_probs.max())


Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
TabM train probs min/mean/max: 0.0005119745 0.25603554 0.842113
XGB  train probs min/mean/max: 0.0034326005 0.20450807 0.8821628
CB   train probs min/mean/max: 0.0036373014143064883 0.3419226216986207 0.9449020215461476


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


## Benchmark Models

In [11]:
def model_best_f1(probs, name, thresholds=None):
    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)
    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(y_train, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    print(f"{name}: best F1 = {best_f1:.6f} at threshold {best_t:.3f}")
    return best_f1, best_t

In [12]:


# You already computed these earlier:
# tabm_train_probs = tabm_predict_proba(df)
# xgb_train_probs  = xgb_predict_proba(df)
# cb_train_probs   = cb_predict_proba(df)

tabm_f1, tabm_t = model_best_f1(tabm_train_probs, "TabM")
xgb_f1,  xgb_t  = model_best_f1(xgb_train_probs,  "XGB")
cb_f1,   cb_t   = model_best_f1(cb_train_probs,   "CatBoost")

TabM: best F1 = 0.598859 at threshold 0.300
XGB: best F1 = 0.617929 at threshold 0.300
CatBoost: best F1 = 0.608145 at threshold 0.500


## Optuna for Weights

In [13]:
def compute_best_f1(probs, targets, thresholds=None):
    probs = np.asarray(probs)
    targets = np.asarray(targets).astype(int)

    if thresholds is None:
        thresholds = np.linspace(0.05, 0.95, 19)

    best_f1, best_t = 0.0, 0.5
    for t in thresholds:
        preds = (probs >= t).astype(int)
        f1 = f1_score(targets, preds, zero_division=0)
        if f1 > best_f1:
            best_f1, best_t = f1, t
    return best_f1, best_t

In [14]:
def blend3_objective(trial):
    # Raw weights; TabM must be >= 0.5 before normalization
    w_tabm_raw = trial.suggest_float("w_tabm_raw", 0.5, 1.0)
    w_xgb_raw  = trial.suggest_float("w_xgb_raw", 0.0, 1.0)
    w_cb_raw   = trial.suggest_float("w_cb_raw",  0.0, 1.0)

    w_sum = w_tabm_raw + w_xgb_raw + w_cb_raw
    if w_sum == 0:
        w_tabm = w_xgb = w_cb = 1.0 / 3.0
    else:
        w_tabm = w_tabm_raw / w_sum
        w_xgb  = w_xgb_raw  / w_sum
        w_cb   = w_cb_raw   / w_sum

    # Blended probabilities on TRAIN
    blend_probs = (
        w_tabm * tabm_train_probs
        + w_xgb  * xgb_train_probs
        + w_cb   * cb_train_probs
    )

    best_f1, best_t = compute_best_f1(blend_probs, y_train)

    # Log useful info
    trial.set_user_attr("w_tabm", w_tabm)
    trial.set_user_attr("w_xgb",  w_xgb)
    trial.set_user_attr("w_cb",   w_cb)
    trial.set_user_attr("best_threshold", best_t)

    return best_f1

In [20]:
blend3_study = optuna.create_study(
    direction="maximize",
    study_name="tabm_xgb_cb_blend3_tabm_anchor",
)
blend3_study.optimize(blend3_objective, n_trials=250, show_progress_bar=True)

best_trial = blend3_study.best_trial
best_f1 = best_trial.value
best_w_tabm = best_trial.user_attrs["w_tabm"]
best_w_xgb  = best_trial.user_attrs["w_xgb"]
best_w_cb   = best_trial.user_attrs["w_cb"]
best_threshold = best_trial.user_attrs["best_threshold"]

print("\n=== Best 3-model blend on TRAIN ===")
print(f"Best F1 (blend): {best_f1:.6f}")
print(f"Weights: TabM={best_w_tabm:.3f}, XGB={best_w_xgb:.3f}, CB={best_w_cb:.3f}")
print(f"Best threshold: {best_threshold:.4f}")


[I 2025-11-14 13:04:15,618] A new study created in memory with name: tabm_xgb_cb_blend3_tabm_anchor
Best trial: 4. Best value: 0.616391:   2%|▏         | 6/250 [00:00<00:07, 31.01it/s]

[I 2025-11-14 13:04:15,659] Trial 0 finished with value: 0.6061577934573444 and parameters: {'w_tabm_raw': 0.7844237369919351, 'w_xgb_raw': 0.03694619631376872, 'w_cb_raw': 0.48807743454794583}. Best is trial 0 with value: 0.6061577934573444.
[I 2025-11-14 13:04:15,690] Trial 1 finished with value: 0.6106658116917816 and parameters: {'w_tabm_raw': 0.5849772762983101, 'w_xgb_raw': 0.4497804159131533, 'w_cb_raw': 0.7775390727487602}. Best is trial 1 with value: 0.6106658116917816.
[I 2025-11-14 13:04:15,720] Trial 2 finished with value: 0.6118869762910035 and parameters: {'w_tabm_raw': 0.6680947248272401, 'w_xgb_raw': 0.7256389084241734, 'w_cb_raw': 0.30078404400443537}. Best is trial 2 with value: 0.6118869762910035.
[I 2025-11-14 13:04:15,749] Trial 3 finished with value: 0.6093596596487401 and parameters: {'w_tabm_raw': 0.622885865975028, 'w_xgb_raw': 0.536379579534679, 'w_cb_raw': 0.7280394218129257}. Best is trial 2 with value: 0.6118869762910035.
[I 2025-11-14 13:04:15,780] Trial 4

Best trial: 12. Best value: 0.616707:   5%|▍         | 12/250 [00:00<00:08, 29.70it/s]

[I 2025-11-14 13:04:15,874] Trial 7 finished with value: 0.608170117515389 and parameters: {'w_tabm_raw': 0.8419416090025045, 'w_xgb_raw': 0.5696458790929252, 'w_cb_raw': 0.13503958872171828}. Best is trial 4 with value: 0.6163913991897787.
[I 2025-11-14 13:04:15,925] Trial 8 finished with value: 0.6086387434554974 and parameters: {'w_tabm_raw': 0.7347002519492082, 'w_xgb_raw': 0.4637590768817429, 'w_cb_raw': 0.7235993267063117}. Best is trial 4 with value: 0.6163913991897787.
[I 2025-11-14 13:04:15,957] Trial 9 finished with value: 0.6098901098901099 and parameters: {'w_tabm_raw': 0.6199065665502776, 'w_xgb_raw': 0.19182683790228394, 'w_cb_raw': 0.6113437338446992}. Best is trial 4 with value: 0.6163913991897787.
[I 2025-11-14 13:04:15,988] Trial 10 finished with value: 0.6111055124458329 and parameters: {'w_tabm_raw': 0.9739532848049378, 'w_xgb_raw': 0.9913272118993517, 'w_cb_raw': 0.9804241302566353}. Best is trial 4 with value: 0.6163913991897787.
[I 2025-11-14 13:04:16,019] Trial 

Best trial: 12. Best value: 0.616707:   8%|▊         | 20/250 [00:00<00:07, 31.56it/s]

...

[I 2025-11-14 13:04:23,402] Trial 233 finished with value: 0.6175182481751825 and parameters: {'w_tabm_raw': 0.5089717540005908, 'w_xgb_raw': 0.9458711598133188, 'w_cb_raw': 0.06646488572878588}. Best is trial 167 with value: 0.6180374124777929.
[I 2025-11-14 13:04:23,438] Trial 234 finished with value: 0.6173072883847931 and parameters: {'w_tabm_raw': 0.5225362927928314, 'w_xgb_raw': 0.9437339845996028, 'w_cb_raw': 0.03930673481060043}. Best is trial 167 with value: 0.6180374124777929.
[I 2025-11-14 13:04:23,473] Trial 235 finished with value: 0.616171100678143 and parameters: {'w_tabm_raw': 0.5165108715421148, 'w_xgb_raw': 0.5685448002983851, 'w_cb_raw': 0.015393717156163517}. Best is trial 167 with value: 0.6180374124777929.
[I 2025-11-14 13:04:23,507] Trial 236 finished with value: 0.6175796709018955 and parameters: {'w_tabm_raw': 0.5249279694631227, 'w_xgb_raw': 0.8907746979294432, 'w_cb_raw': 0.06090567913478182}. Best is trial 167 with value: 0.6180374124777929.
[I 2025-11-14 13

Best trial: 237. Best value: 0.618151:  98%|█████████▊| 244/250 [00:08<00:00, 29.63it/s]

[I 2025-11-14 13:04:23,605] Trial 239 finished with value: 0.6169440887750747 and parameters: {'w_tabm_raw': 0.5018762863487041, 'w_xgb_raw': 0.9586285579697328, 'w_cb_raw': 0.0015756666215161474}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,638] Trial 240 finished with value: 0.61775247733502 and parameters: {'w_tabm_raw': 0.5097335987955517, 'w_xgb_raw': 0.9599633012443933, 'w_cb_raw': 0.03153132855539588}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,673] Trial 241 finished with value: 0.6178176067474961 and parameters: {'w_tabm_raw': 0.508383902580523, 'w_xgb_raw': 0.9560777384453052, 'w_cb_raw': 0.030927109576060888}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,707] Trial 242 finished with value: 0.6113266097750194 and parameters: {'w_tabm_raw': 0.507412677910452, 'w_xgb_raw': 0.9621321219264853, 'w_cb_raw': 0.26930750038446044}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:

Best trial: 237. Best value: 0.618151: 100%|██████████| 250/250 [00:08<00:00, 30.06it/s]

[I 2025-11-14 13:04:23,806] Trial 245 finished with value: 0.6175756936385695 and parameters: {'w_tabm_raw': 0.5000493275364357, 'w_xgb_raw': 0.9341099664295415, 'w_cb_raw': 0.028595993223530427}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,841] Trial 246 finished with value: 0.6169873498458595 and parameters: {'w_tabm_raw': 0.5122766150238396, 'w_xgb_raw': 0.9708907516398184, 'w_cb_raw': 0.013139227086812765}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,873] Trial 247 finished with value: 0.6179255207237534 and parameters: {'w_tabm_raw': 0.5156155234622011, 'w_xgb_raw': 0.910659777733133, 'w_cb_raw': 0.031239469237306625}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:04:23,904] Trial 248 finished with value: 0.6174708293913592 and parameters: {'w_tabm_raw': 0.516553928241068, 'w_xgb_raw': 0.9086868503111667, 'w_cb_raw': 0.032486592885696}. Best is trial 237 with value: 0.6181512251551162.
[I 2025-11-14 13:

In [21]:
# Diagnostic Code

best_trial = blend3_study.best_trial
best_w_tabm = best_trial.user_attrs["w_tabm"]
best_w_xgb  = best_trial.user_attrs["w_xgb"]
best_w_cb   = best_trial.user_attrs["w_cb"]

# Rebuild blend_probs for the best weights
blend_probs_best = (
    best_w_tabm * tabm_train_probs
    + best_w_xgb * xgb_train_probs
    + best_w_cb  * cb_train_probs
)

# Scan a fine grid of thresholds
th_grid = np.linspace(0.05, 0.95, 181)  # step 0.005
f1_vals = []

for t in th_grid:
    preds = (blend_probs_best >= t).astype(int)
    f1 = f1_score(y_train, preds, zero_division=0)
    f1_vals.append(f1)

# Show top 10 thresholds
top_idx = np.argsort(f1_vals)[-10:][::-1]
print("Top 10 thresholds for BLEND:")
for i in top_idx:
    print(f"t={th_grid[i]:.3f}, F1={f1_vals[i]:.6f}")


Top 10 thresholds for BLEND:
t=0.300, F1=0.618151
t=0.295, F1=0.617387
t=0.310, F1=0.617159
t=0.305, F1=0.616205
t=0.315, F1=0.615619
t=0.290, F1=0.615195
t=0.285, F1=0.613839
t=0.320, F1=0.612925
t=0.280, F1=0.612024
t=0.325, F1=0.611885


In [17]:
real_test = pd.read_csv("data/Testing_TriGuard.csv")  # if not already loaded

tabm_test_probs = tabm_predict_proba(real_test)
xgb_test_probs  = xgb_predict_proba(real_test)
cb_test_probs   = cb_predict_proba(real_test)

print("TabM test probs min/mean/max:",
      tabm_test_probs.min(), tabm_test_probs.mean(), tabm_test_probs.max())
print("XGB  test probs min/mean/max:",
      xgb_test_probs.min(), xgb_test_probs.mean(), xgb_test_probs.max())
print("CB   test probs min/mean/max:",
      cb_test_probs.min(), cb_test_probs.mean(), cb_test_probs.max())


Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
TabM test probs min/mean/max: 0.0007758692 0.25367633 0.82720715
XGB  test probs min/mean/max: 0.0035788997 0.20258325 0.8522331
CB   test probs min/mean/max: 0.0035900861756748156 0.3395900545022105 0.9450703883041788


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/validation.py:2742: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [18]:
blend_test_probs = (
    best_w_tabm * tabm_test_probs
    + best_w_xgb * xgb_test_probs
    + best_w_cb  * cb_test_probs
)

blend_test_preds = (blend_test_probs >= best_threshold).astype(int)


In [19]:
ensemble_pred = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": blend_test_preds,
})

out_path = "results/tabm_xgb_cb_tabmAnchor_optuna_blend.csv"
ensemble_pred.to_csv(out_path, index=False)
print("Saved:", out_path)

Saved: results/tabm_xgb_cb_tabmAnchor_optuna_blend.csv
